## Experiment Setup

### Random seed / PyTorch / CUDA related

In [1]:
import time
import datetime
import os
import sys
import itertools

# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')
    
    # If there's a package I need to install separately, do it here
    !pip install pyro-ppl

    # cd to the appropriate working directory under my Google Drive
    %cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
    
    # List the directory contents
    !ls

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 682

### Third party libraries (NumPy, PyTorch, Pyro)

In [2]:
# Third party libraries import
import numpy as np
import torch
import pyro
import matplotlib.pyplot as plt

# Print version information
print("Python Version: " + sys.version)
print("NumPy Version: " + np.__version__)
print("PyTorch Version: " + torch.__version__)
print("Pyro Version: " + pyro.__version__)

Python Version: 3.8.0 (v3.8.0:fa919fdf25, Oct 14 2019, 10:23:27) 
[Clang 6.0 (clang-600.0.57)]
NumPy Version: 1.17.4
PyTorch Version: 1.4.0a0+328ec54
Pyro Version: 1.0.0


In [3]:
# More imports...
from torch import nn, optim
from torch.utils.data import random_split, DataLoader, RandomSampler
import torchvision
import torchvision.transforms as transforms
from pyro.infer import SVI, Trace_ELBO, HMC, MCMC

# Import model and dataset classes from ronald_bdl
from ronald_bdl import models, datasets

# pyplot setting
%matplotlib inline

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')

    torch.backends.cudnn.deterministic = True
    
    # Disable 'benchmark' mode
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = False
    use_pin_memory = True # Faster Host to GPU copies with page-locked memory

    # CUDA libraries version information
    print("CUDA Version: " + str(torch.version.cuda))
    print("cuDNN Version: " + str(torch.backends.cudnn.version()))
    print("CUDA Device Name: " + str(torch.cuda.get_device_name()))
    print("CUDA Capabilities: "+ str(torch.cuda.get_device_capability()))
else:
    torch_device = torch.device('cpu')
    use_pin_memory = False

### Variable settings

#### Data prep

In [4]:
# CIFAR10 data transformation setting
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Set the proportion of the original dataset to be available as a whole
subset_proportions = [0.05]

# Proportion of the dataset to be used for training
dataset_train_size = 0.8

# Number of dataset splits
n_splits = 10

#### NN settings

In [5]:
# Dropout
dropout_rates = [0.1, 0.3, 0.5]

# Regularization strengths
reg_strengths = [0.05]


### Training setup

In [6]:
# Epochs
n_epochs = [40, 400, 4000]

# Optimizer learning rate
optimizer_learning_rate = 0.001 # PyTorch default value is 0.001

# Data batch sizes
n_training_batch = 512

# Number of test predictions (for each data point)
n_prediction = 500

# Cross Entropy to minimize
objective = nn.CrossEntropyLoss()

# Test start time
test_start_time = datetime.datetime.today().strftime('%Y%m%d%H%M')

print(test_start_time)

201912030001


## Train the network

In [7]:
for subset_prop, dropout_rate, reg_strength, n_epoch in itertools.product(
    subset_proportions,
    dropout_rates, reg_strengths, 
    n_epochs,
):  
    # Reset the random number generator for each method (to produce identical results)
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    pyro.set_rng_seed(random_seed)

    # Print parameter combinations being tested
    print(
        "subset %f, dropout_rate %f, reg_strength %f"
        % (subset_prop, dropout_rate, reg_strength))

    print("n_epoch %d" % n_epoch)
    print()

    """
    Results file storage
    """

    # Create directory to store results for the current test configuration
    test_results_path = os.path.join(
        './test_results',
        'error_convergence_2',
        'CIFAR-10',
        test_start_time,
        (
            str(subset_prop)
            + '_' + str(dropout_rate) 
            + '_' + str(reg_strength)
            + '_' + str(n_epoch)),
    )

    os.makedirs(test_results_path, exist_ok=True)

    test_results_accuracy_mc_path = os.path.join(
        test_results_path,
        "accuracy_mc.txt"
    )

    test_results_accuracy_non_mc_path = os.path.join(
        test_results_path,
        "accuracy_non_mc.txt"
    )

    test_results_lls_mc_path = os.path.join(
        test_results_path,
        "lls_mc.txt"
    )

    """
    Dataset multiple splits prep
    """
    # Prepare new subset of the original dataset
    subset = datasets.CIFAR10(
        root='./datasets_files', limit_size=subset_prop, transform=transform, download=True)

    # Determine sizes of training and testing set
    train_size = int(dataset_train_size * len(subset))
    test_size = len(subset) - train_size

    # Print the size of the subset
    print("subset size = " + str(subset.data.shape))
    print("training set size = %d" % train_size)
    print("test set size = %d" % test_size)
    print()

    # Prepare multiple sets of random train-test splits 
    # to test the parameter combination
    subset_splits = []

    for _ in range(n_splits):
        train, test = random_split(subset, lengths=[train_size, test_size])
        subset_splits.append((train, test))

    """
    Training & testing
    """

    # Try learning with different splits
    for s, (train, test) in enumerate(subset_splits):

        """
        Training
        """

        print('Training with split %d' % s)

        train_loader = DataLoader(train, batch_size=n_training_batch, pin_memory=use_pin_memory)

        # Prepare network
        network = models.SimpleCIFAR10(
            dropout_rate=dropout_rate,
            dropout_type='bernoulli',
        )

        # Send the whole model to the selected torch.device
        network.to(torch_device)

        # Model to train mode
        network.train()

        # Adam optimizer
        # https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam
        # NOTE: Need to set L2 regularization from here
        optimizer = optim.Adam(
            network.parameters(),
            lr=optimizer_learning_rate,
            weight_decay=reg_strength, # L2 regularization
        )

        # Record training start time (for this split)
        tic = time.time()

        for epoch in range(n_epoch): # loop over the dataset multiple times
            # Mini-batches
            for data in train_loader:
                # get the inputs; data is a list of [inputs, labels]
                inputs, targets = data

                # Store the batch to torch_device's memory
                inputs = inputs.to(torch_device)
                targets = targets.to(torch_device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = network(inputs)

                loss = objective(outputs, targets)
                loss.backward()

                optimizer.step()

            print("epoch %d loss = %f" % (epoch, loss.item()))

        # Record training end time
        toc = time.time()

        # Report the final loss
        print("final loss = %f" % (loss.item()))

        """
        Testing
        """

        # Model to eval mode
        network.eval()

        # Store the batch to torch_device's memory
        test_loader = DataLoader(test, batch_size=n_training_batch, pin_memory=use_pin_memory)

        # Record testing start time
        tic_testing = time.time()

        _, mean, metrics = network.predict_dist(test_loader, n_prediction)

        # Record testing end time
        toc_testing = time.time()

        # Record all the scores to the score files
        if len(metrics) > 0:
            for key, value in metrics.items():
                print(str(key) + " = " + str(value))

                if key == 'accuracy_mc':
                    with open(test_results_accuracy_mc_path, 'a+') as accuracy_mc_file:
                        accuracy_mc_file.write('%d %f \n' % (s, value))

                elif key == 'accuracy_non_mc':
                    with open(test_results_accuracy_non_mc_path, 'a+') as accuracy_non_mc_file:
                        accuracy_non_mc_file.write('%d %f \n' % (s, value))

                elif key == 'test_ll_mc':
                    with open(test_results_lls_mc_path, 'a+') as lls_mc_file:
                        lls_mc_file.write('%d %f \n' % (s, value))

        # Report the total training time
        print("training time = " + str(toc - tic) + " seconds")

        # Report the total testing time
        print("testing time = " + str(toc_testing - tic_testing) + " seconds")

        print()

subset 0.050000, dropout_rate 0.100000, reg_strength 0.050000
n_epoch 10

Files already downloaded and verified
subset size = (2500, 32, 32, 3)
training set size = 2000
test set size = 500

Training with split 0


/Users/se9/python/00_default/lib/python3.8/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


epoch 0 loss = 1.990959
epoch 1 loss = 1.953834
epoch 2 loss = 2.058130
epoch 3 loss = 2.052957
epoch 4 loss = 1.966764
epoch 5 loss = 2.132823
epoch 6 loss = 2.122897
epoch 7 loss = 2.056861
epoch 8 loss = 2.096857
epoch 9 loss = 1.989916
final loss = 1.989916
accuracy_mc = tensor(0.2659)
accuracy_non_mc = tensor(0.2549)
test_ll_mc = tensor(-1.9555)
training time = 17.652841329574585 seconds
testing time = 45.73752522468567 seconds

Training with split 1
epoch 0 loss = 2.228805
epoch 1 loss = 2.091846
epoch 2 loss = 2.089999
epoch 3 loss = 2.041809
epoch 4 loss = 2.107483
epoch 5 loss = 2.080072
epoch 6 loss = 2.163190
epoch 7 loss = 1.839329
epoch 8 loss = 1.994308
epoch 9 loss = 2.001613
final loss = 2.001613
accuracy_mc = tensor(0.2216)
accuracy_non_mc = tensor(0.2248)
test_ll_mc = tensor(-1.8954)
training time = 19.068665981292725 seconds
testing time = 43.475486040115356 seconds

Training with split 2
epoch 0 loss = 2.105234
epoch 1 loss = 2.071286
epoch 2 loss = 2.055319
epoch 3

KeyboardInterrupt: 

## Results visualization

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

experiment_root_directory = os.path.join(
    './test_results',
    'error_convergence_2',
    'CIFAR-10',
    '201912011915',
)

for subset_prop, dropout_rate, reg_strength, n_epoch in itertools.product(
    subset_proportions, dropout_rates, reg_strengths, n_epochs
):
    for metric_name in ['lls_mc', 'accuracy_mc', 'accuracy_non_mc']:
        figure_file_name = (
            '3d_' + str(subset_prop)
            + '_' + str(dropout_rate)
            + '_' + str(reg_strength)
            + '_' + str(n_epoch)
            + '_' + metric_name + '.png'
        )
        
        figure_title = (
            metric_name 
            + (' subset %f, dropout rate = %f, reg strength = %f, n_epoch = %d' % (subset_prop, dropout_rate, reg_strength, n_epoch))
        )

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        fig.tight_layout(pad=2, rect=[0, 0.00, 2, 2])
        
        hidden_dims_data = []
        hidden_layers_data = []
        scores_data = []
        
        for hidden_dim in network_hidden_dims:
            for n_layer in network_hidden_layers:
                # Open the score file
                score_file_path = os.path.join(
                    experiment_root_directory,
                    (
                        str(subset_prop) 
                        + '_' + str(hidden_dim)
                        + '_' + str(n_layer) 
                        + '_' + str(dropout_rate) 
                        + '_' + str(reg_strength)
                        + '_' + str(n_epoch)
                    ),
                    metric_name + '.txt',
                )

                scores = np.loadtxt(score_file_path).T
                
                for s in scores[1]:
                    # Multiple values (for each split) for
                    # each (hidden_dim, n_layer) combination
                    hidden_dims_data.append(hidden_dim)
                    hidden_layers_data.append(n_layer)
                    scores_data.append(s)
                    
                mean = np.mean(scores[1])
                var = np.var(scores[1])

        ax.set_xlabel('hidden layer units')
        ax.set_ylabel('number of hidden layers')
        
        #if metric_name in ('rmse_mc', 'rmse_non_mc'):
        #    ax.set_zlim([5, 20])
        #elif metric_name == 'lls_mc':
        #    ax.set_zlim([-10, 0])
    
        ax.scatter3D(hidden_dims_data, hidden_layers_data, scores_data, c=scores_data)

        fig.suptitle(figure_title, y=2.05)        
        
        plt.savefig(
            os.path.join(experiment_root_directory, figure_file_name),
            dpi=600,
            bbox_inches='tight',
        )
        
        plt.show()